<a href="https://colab.research.google.com/github/deoashokanand/DSBA-6345-Spring-2022/blob/main/Ashok_Gensim2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re 
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

2022-03-24 09:45:43,603 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 22.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=0a9c24dd411a7afd6dfd1baee8f7677e62be2dc6e5d8031e60296cad2929ea69
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [3]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

2022-03-24 09:46:18,972 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2022-03-24 09:46:19,026 : INFO : Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive


In [4]:
df = pd.read_csv('/content/Telecommunication455.csv', encoding = 'unicode_escape')
print (df.shape)
df.tail()

/usr/lib/python3.7/encodings/unicode_escape.py:26: DeprecationWarning: invalid escape sequence '\-'
  return codecs.unicode_escape_decode(input, self.errors)[0]


(15000, 10)


,PatentId,DatePublished,Assignee,Title,City,State,AbstractText,usClaimStatement,SubClass,FurtherClassification
14995,07596393,20090929,Electronics and Telecommunications Research In...,Wearable mobile phone using EMG and controllin...,Taejon,NaN,Disclosed are a wearable mobile phone capable ...,What is claimed is:,575.1,455/575.2 455/ 90.1
14996,07596394,20090929,Lite-On It Corporation,Handhold wireless device,Taipei,NaN,When the frame is slid to the second end the ...,What is claimed is:,575.1,455/ 90.3 379/325 379/329
14997,07596395,20090929,Nokia Corporation,Depressible hinge and mobile stations using same,Espoo,NaN,The present invention is directed to an openin...,That which is claimed:,575.3,455/ 90.3 455/575.1 379/433.13
14998,07596396,20090929,Sony Ericsson Mobile Communications AB,Flexible gaskets for wireless terminals with s...,Lund,NaN,Gaskets and wireless terminals with gaskets fo...,That which is claimed is:,575.4,455/575.1
14999,07606524,20091020,"Rockwell Collins, Inc.",Integrated monitoring and communications recei...,Cedar Rapids,IA,The present invention is an integrated monitor...,The invention claimed is:,1,455/456.1 455/410


# New Section

In [5]:
# Convert to list
df['AbstractText'] = df['AbstractText'].apply(str)
data = df.AbstractText.values.tolist()
data_words = list(data)
print(data_words[:1])



["A system  apparatus  method and computer program product to obtain comprehensive vehicle radio listener statistics based on parameters such as radio status (e.g.  on/off status and CD/Tape/AM/FM setting)  radio volume  station preset information  current frequency setting (i.e.  station identification)  and Global Positioning Satellite (GPS) system coordinates is disclosed. A vehicle-mounted field unit for collecting and transmitting such parameters to a base station is also disclosed. The system monitors and stores all events related to the occupants' interaction with the vehicle's radio  including automatic detection of the selected radio station through a speaker port. The stored data is then transmitted to a base station's central collection computer for immediate compilation and analysis. The system is capable of producing detailed reports containing error-free  unbiased  audience measurement statistics which can be made available to subscribers such as broadcasters  corporate a

In [6]:
#Create Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords



stop_words = stopwords.words ('english')
stop_words.extend(['from', '.',',','the','a','of','to','and','is', 'in', 'for',
                                  'an', ')', '(', 'with', 'or', 'by', 'first', 'from', 'that', 'on', 'to', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'na', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'br', 'sub', 'mech','non'])

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in data_words]

In [8]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

2022-03-24 09:47:45,576 : INFO : collecting all words and their counts
2022-03-24 09:47:45,580 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-03-24 09:47:46,970 : INFO : PROGRESS: at sentence #10000, processed 687602 words and 269732 word types
2022-03-24 09:47:47,672 : INFO : collected 353060 word types from a corpus of 1025419 words (unigram + bigrams) and 15000 sentences
2022-03-24 09:47:47,674 : INFO : using 353060 counts as vocab in Phrases<0 vocab, min_count=20, threshold=10.0, max_vocab_size=40000000>
/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [10]:
# Remove rare and common tokens.
from gensim import corpora

# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2022-03-24 09:47:56,801 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-03-24 09:47:57,430 : INFO : adding document #10000 to Dictionary(12104 unique tokens: ['advertiser', 'advertising', 'agency', 'analysis', 'apparatus']...)
2022-03-24 09:47:57,722 : INFO : built Dictionary(13704 unique tokens: ['advertiser', 'advertising', 'agency', 'analysis', 'apparatus']...) from 15000 documents (total 1080961 corpus positions)
2022-03-24 09:47:57,767 : INFO : discarding 10585 tokens: [('agency', 14), ('broadcaster', 8), ('compilation', 3), ('comprehensive', 8), ('corporate', 7), ('immediate', 17), ('occupant', 7), ('tape', 12), ('unbiased', 3), ('discrimination', 14)]...
2022-03-24 09:47:57,769 : INFO : keeping 3119 tokens which were in no less than 20 and no more than 7500 (=50.0%) documents
2022-03-24 09:47:57,779 : INFO : resulting dictionary: Dictionary(3119 unique tokens: ['advertiser', 'advertising', 'analysis', 'apparatus', 'audience']...)


In [11]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [12]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 3119
Number of documents: 15000


In [13]:
# Train LDA Model and this is going to take few minutes
from gensim.models import LdaModel

# Set training parameters.
num_topics = 25
chunksize = 2000
passes = 20
iterations = 4
eval_every = None  # Takes too much time to do 

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2022-03-24 09:48:16,496 : INFO : using autotuned alpha, starting with [0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]
2022-03-24 09:48:16,500 : INFO : using serial LDA version on this node
2022-03-24 09:48:16,543 : INFO : running online (multi-pass) LDA training, 25 topics, 20 passes over the supplied corpus of 15000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 4x with a convergence threshold of 0.001000
2022-03-24 09:48:16,552 : INFO : PROGRESS: pass 0, at document #2000/15000
2022-03-24 09:48:17,246 : INFO : optimized alpha [0.07623321, 0.076318376, 0.076123044, 0.07628104, 0.076323904, 0.07629358, 0.0764437, 0.07629802, 0.07613969, 0.07622832, 0.07645723, 0.07630092, 0.076241374, 0.07635755, 0.07635558, 0.07631235, 0.076005965, 0.07636993, 0.07629235, 0.076189786, 0.076399334, 0.076310165, 0.07625051, 0.07641195, 0.07635064

In [14]:
print('\nPerplexity: ', model.log_perplexity(corpus)) 

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us


Perplexity:  -6.218184332694436


In [15]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-03-24 09:49:39,785 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-03-24 09:49:39,813 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-03-24 09:49:39,840 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-03-24 09:49:39,866 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-03-24 09:49:39,894 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-03-24 09:49:39,920 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-03-24 09:49:39,948 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-03-24 09:49:39,974 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-03-24 09:49:40,000 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-03-24 09:49:40,030 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-03-24 09:49:40,056 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-03-24 09:49:40

Average topic coherence: -3.1653.
[([(0.29740134, 'communication'),
   (0.1468574, 'terminal'),
   (0.07593739, 'mobile'),
   (0.045787547, 'apparatus'),
   (0.03721447, 'system'),
   (0.035063718, 'second'),
   (0.022091618, 'mean'),
   (0.020551315, 'operation'),
   (0.015555176, 'method'),
   (0.01020664, 'function'),
   (0.009012281, 'via'),
   (0.008893866, 'setting'),
   (0.008234339, 'using'),
   (0.00813594, 'predetermined'),
   (0.007462006, 'provided'),
   (0.006532583, 'according'),
   (0.0060305516, 'includes'),
   (0.0058596935, 'type'),
   (0.005790534, 'capable'),
   (0.0057856394, 'performing')],
  -2.2630241015944237),
 ([(0.17929445, 'unit'),
   (0.1573559, 'radio'),
   (0.07650698, 'control'),
   (0.05661375, 'receiving'),
   (0.05299517, 'module'),
   (0.05140929, 'processing'),
   (0.029315492, 'transmitting'),
   (0.029064007, 'processor'),
   (0.023256026, 'includes'),
   (0.021313589, 'receiver'),
   (0.0145912925, 'field'),
   (0.013167353, 'controller'),
   (0

In [16]:
!pip install pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18     0.130064 -0.246116       1        1  6.046007
8     -0.078772  0.000525       2        1  5.809101
6      0.016062  0.065893       3        1  5.240370
15     0.284574 -0.031688       4        1  5.051679
10    -0.076239  0.093880       5        1  5.047787
17    -0.070092 -0.022691       6        1  4.858554
16    -0.148179 -0.085677       7        1  4.839957
19    -0.091301 -0.029778       8        1  4.818792
11     0.258891 -0.095977       9        1  4.581400
1     -0.198082 -0.032277      10        1  4.324490
12     0.129675 -0.039655      11        1  4.305431
5     -0.068724 -0.115505      12        1  4.305028
24     0.109364 -0.148864      13        1  3.956556
2      0.220976 -0.031818      14        1  3.955115
22    -0.125686 -0.084801      15        1  3.637221
14    -0.092127  0.023107      16        1  3.479110
23    -0.117364 -0.035530      17        1  3.397510
4      0.011982 -0.000274      18        1  3.327612
0      0.149012  0.246736      19        1  3.286681
9     -0.109568  0.110276      20        1  3.081781
7     -0.133295 -0.064478      21        1  2.915191
3      0.035160  0.233591      22        1  2.726439
21     0.069443  0.211335      23        1  2.500215
13    -0.098082 -0.009774      24        1  2.489888
20    -0.007691  0.089559      25        1  2.018085, topic_info=               Term          Freq         Total Category  logprob  loglift
174   communication  17570.000000  17570.000000  Default  30.0000  30.0000
254          device  14837.000000  14837.000000  Default  29.0000  29.0000
170          signal  23216.000000  23216.000000  Default  28.0000  28.0000
59          station  11293.000000  11293.000000  Default  27.0000  27.0000
486        wireless  12284.000000  12284.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
1235        optical    113.577893    146.943676  Topic25  -5.1968   3.6455
569          reader    129.766111    212.482723  Topic25  -5.0635   3.4099
208        external    182.575697    748.380101  Topic25  -4.7221   2.4923
176      controller    196.985996   2012.045773  Topic25  -4.6461   1.5792
158         include    152.355859   1393.727137  Topic25  -4.9031   1.6895

[1351 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2862     18  0.998743  abstract
520       5  0.621843    access
520       6  0.033065    access
520      10  0.158656    access
520      15  0.092804    access
...     ...       ...       ...
2698      2  0.993319      worn
1739     25  0.995579    writer
3027     16  0.995219      wtru
296       7  0.767521      zone
296      17  0.231308      zone

[3016 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 9, 7, 16, 11, 18, 17, 20, 12, 2, 13, 6, 25, 3, 23, 15, 24, 5, 1, 10, 8, 4, 22, 14, 21])

In [17]:
print('\nPerplexity: ', model.log_perplexity(corpus)) 

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us


Perplexity:  -6.218203248818867
